In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import GatheringModel, read_input, compute_multiphase_pressure_drop, compute_gas_pressure_drop
import pandas as pd
from gamspy import Options, Ord

In [2]:
m = GatheringModel("Prototipo", data= read_input("toy_problem.xlsx"))

--- Job _lwNvpqsnQLicgl2iP9tTwQ.gms Start 09/16/25 15:37:58 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\tmpna8qc67b\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\
    LimRow 0
    LimCol 0
    TraceOpt 3
    SolPrint 0


In [ ]:
m.solution_algorithm(scenario_name="toy_problem", max_iterations=2, solver_opts={"solver": "gurobi", "gap": 0.0001, "max_time": 300.0})

--- Iteration 1 ---
--- _lwNvpqsnQLicgl2iP9tTwQ.gms(1039) 11 Mb
--- Job _lwNvpqsnQLicgl2iP9tTwQ.gms Start 09/16/25 15:39:34 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\tmpna8qc67b\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\

[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


--- _lwNvpqsnQLicgl2iP9tTwQ.gms(1109) 159 Mb
--- Job _lwNvpqsnQLicgl2iP9tTwQ.gms Start 09/16/25 15:45:02 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\tmpna8qc67b\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\
    LimRow 

[CONVERTER - INFO] GAMS (.gms) file has been generated under toy_problem/iteration_1\Multiphase_network_design.gms


--- Iteration 2 ---
--- _lwNvpqsnQLicgl2iP9tTwQ.gms(11) 159 Mb
--- Job _lwNvpqsnQLicgl2iP9tTwQ.gms Start 09/16/25 15:47:04 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\tmpna8qc67b\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\C

[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


--- _lwNvpqsnQLicgl2iP9tTwQ.gms(1261) 194 Mb
--- Job _lwNvpqsnQLicgl2iP9tTwQ.gms Start 09/16/25 15:52:31 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\tmpna8qc67b\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\_lwNvpqsnQLicgl2iP9tTwQ.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpbc7azezw\
    LimRow 

In [4]:
compute_multiphase_pressure_drop(Qoil=6500, Qgas=13000, Qwater=22750, p_inlet=1.3664*250, dist=1.25623, diam=12)

(0.2951863996284128,
 0.20658700991114926,
 8.518578996446843,
 0.007158667624155068,
 0.03465207046287146)

In [27]:
1.167/(0.25*1.6)

2.9175

In [8]:
from math import pi, sqrt

In [10]:
mpa_to_psi = 145.038
m3_to_cf = 35.3147
bbl_to_m3 = 0.158987
in_to_m = 0.0254
mile_to_km = 1.60934
hffl = 0.025
rho_liq = 850
n = 4.12

qoil = 6500
qwater = 22750
qgas = 13000
p_inlet = 1.16756
dist = 0.25
diam = 8
kw_gas = 1.3076

### Compute dpLiq
vel_liq = ((qoil + qwater)*bbl_to_m3)/((pi*((diam*in_to_m)**2)/4)*24*3600)
dp_liq = ((dist*mile_to_km)*(hffl/2)*(rho_liq*vel_liq*vel_liq)/(diam*in_to_m))/1e3

### Compute dpGas
pressSQ = (p_inlet*1e3)**2
kw_gas = sqrt(0.729*288.9*(0.25*1.6)*1)/(0.0037435*0.92*(288.9/(101.325*1e3))*((20*in_to_m*1e3)**2.667))
aux = (pressSQ - ((qgas*1e3/m3_to_cf)*kw_gas)**2)
print(aux)
pressGAS = sqrt(aux)/1e3
dp_gas = p_inlet - pressGAS

ixlm = dp_gas/dp_liq
ylp = ((ixlm**(1/n))+1)**n
dp = dp_liq*ylp

max_incr = p_inlet - 0.551581
ixlm_inf = dp_gas/(max_incr/ylp + dp_liq)
ylp_inf = ((ixlm_inf**(1/n))+1)**n

-435394426.71507835


ValueError: math domain error

In [27]:
diam = 12
dist = 1.25623

In [30]:
density_GP: float = 0.729
pipe_roughness: float = 0.0001
Tavg: float = 288.9
P0: float = 0.101325
T0: float = 288.9
n: float = 4.12
z = 1
kw:float = 0.0037435
pipe_efficiency: float = 0.92

In [35]:
# kw_gas = (sqrt(density_GP*Tavg*(dist*1.6)*z)/(kw*pipe_efficiency*(T0/(P0))*((diam*in_to_m*1e3)**2.667)))**2
kw_gas = ((sqrt(density_GP*Tavg*(dist*mile_to_km)*z))/(kw*0.92*(T0/(P0*1e3))*((diam*in_to_m*1e3)**2.667)))**2
aux = ((qgas*1e3/m3_to_cf)**2*kw_gas)

In [36]:
p_inlet = 1.366
pressSQ = (p_inlet*1e3)**2

In [37]:
pf = sqrt(pressSQ - aux)

In [38]:
pf/1e3

1.3536197185507808

In [33]:
aux_expr =  ((13000*1e3/m3_to_cf)*sqrt(density_GP*Tavg*(dist*mile_to_km)*z))/(kw*0.92*(T0/(P0*1e3))*((diam*in_to_m*1e3)**2.667))
pf = sqrt(max((1.366*1e3)**2 - aux_expr**2, 0))
pf = pf/1e3 #kPa to MPa

In [34]:
pf

1.3536197185507808

In [37]:
aux

1.3052731598206841

In [17]:
13000/1e3/m3_to_cf

0.36811865880214184

In [16]:
13000*1e3/m3_to_cf

368118.65880214184

In [20]:
aux = (pressSQ*(1e3**2) - ((8800*1e3/m3_to_cf)**2)*0.003223)
aux

-198767915.1099131

In [ ]:
1.7236

1.7236

In [12]:
dp_liq

0.05795167022475664

In [11]:
(pressSQ*(1e3**2) - ((qgas*1e3/m3_to_cf)**2)*0.0042744)

-577866505.084905

In [9]:
pressSQ*(1e3**2)

1363196.3535999998

In [8]:
((qgas*(1e3/m3_to_cf))**2)*kw_gas

177194637282.65704